In [1]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel

torch.random.manual_seed(0)

model_path = "Qwen/Qwen3-4B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype="auto",
)
model = PeftModel.from_pretrained(model, 'qwen3_lora_finetuned_v2/checkpoint-2118/')

tokenizer = AutoTokenizer.from_pretrained(model_path)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [3]:
generation_args = {
    "max_new_tokens": 50,
    "return_full_text": True,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 0.8,
    "top_k": 20,
    "min_p": 0,
}

def get_prompt():
    return [
        {
            "role": "user", 
            "content": "Write a satirical headline about Donald Trump."
        }
    ]

output = pipe(get_prompt(), num_return_sequences=5, **generation_args)
output

[{'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline about Donald Trump.'},
   {'role': 'assistant',
    'content': 'Donald Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump. Trump'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline about Donald Trump.'},
   {'role': 'assistant',
    'content': 'Donald Trump: ‘I Want To Be President’<im_end>\nassistant\nDonald Trump: ‘I Want To Be President’<im_end>\nassistant\nDonald Trump: ‘I Want To Be President’<im_end>\nassistant'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline about Donald Trump.'},
   {'role': 'assistant',
    'content': 'Donald Trump Announces He’s Running For President Again<im_end>\nassistant\nDonald Trump Announces He’s Running For President Again<im_end>\nassistant\nDonald Trump Announces He’s Running For Presid

In [3]:
prompt = get_prompt()
prompt.append({
    "role": "assistant",
    "content": "U.S. To Reopen Embassy In Jerusalem",
})

tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
)

'<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\nU.S. To Reopen Embassy In Jerusalem<|im_end|>\n'

In [4]:
print(tokenizer.chat_template)

{%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0].role == 'system' %}
        {{- messages[0].content + '\n\n' }}
    {%- endif %}
    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0].role == 'system' %}
        {{- '<|im_start|>system\n' + messages[0].content + '<|im_end|>\n' }}
    {%- endif %}
{%- endif %}
{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}
{%- for message in messages[::-1] %}
    {%- set index = (messages|length - 

In [5]:
new_template = '''{% for message in messages %}{{'<|im_start|>' + message['role'] + '\\n' + message['content'] + '<im_end>' + '\\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\\n' }}{% endif %}'''

if tokenizer.chat_template != new_template:
    original_template = tokenizer.chat_template
    tokenizer.chat_template = new_template

tokenizer.apply_chat_template(
    prompt,
    tokenize=False,
)

'<|im_start|>user\nWrite a satirical headline in the style of Onion News. Only write the headline.<im_end>\n<|im_start|>assistant\nU.S. To Reopen Embassy In Jerusalem<im_end>\n'

In [5]:
guided_prompt = [{
    "role": "user", 
    "content": "Write a satirical headline in the style of Onion News about the housing market."
}]

output = pipe(guided_prompt, num_return_sequences=5, **generation_args)
output

[{'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News about the housing market.'},
   {'role': 'assistant',
    'content': 'The Housing Market: A History And Future Forecast<im_end>\nassistant\nThe Housing Market: A History And Future Forecast<im_end>\nassistant\nThe Housing Market: A History And Future Forecast<im_end>\nassistant\nThe Housing'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News about the housing market.'},
   {'role': 'assistant',
    'content': 'The Housing Market: What To Expect In 2022 And Beyond<im_end>\nassistant\nThe Housing Market: What To Expect In 2022 And Beyond<im_end>\nassistant\nThe Housing Market: What To'}]},
 {'generated_text': [{'role': 'user',
    'content': 'Write a satirical headline in the style of Onion News about the housing market.'},
   {'role': 'assistant',
    'content': 'How The Housing Market Works<im_end>\nassistant\nHow The H